# Automated ML

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [52]:
import requests
import json
import logging
import joblib
from pprint import pprint
import pandas as pd
from sklearn.model_selection import train_test_split

import azureml.core
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails
from azureml.core.model import Model, InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.automl.core.shared import constants
from azureml.core.environment import Environment

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.33.0


## Initialize Workspace

Initialize a workspace object from persisted configuration. Make sure the config file is present at ./config.json

In [2]:
ws = Workspace.from_config()

print('Workspace name: ' + ws.name,
      'Azure region: ' + ws.location,
      'Subscription id: ' + ws.subscription_id,
      'Resource group: ' + ws.resource_group, sep='\n')


Workspace name: workspace-rvl
Azure region: westeurope
Subscription id: b17f1c19-34a2-47b8-a207-40ea477828fc
Resource group: resource-group-rvl


## Create an Azure ML experiment
Let's create an experiment named "automl-experiment-heart-failure-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the source directory for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the source directory would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the source_directory of the step.


In [3]:
experiment_name = 'automl-experiment-heart-failure-classification'
project_folder = './classification-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automl-experiment-heart-failure-classification,workspace-rvl,Link to Azure Machine Learning studio,Link to Documentation


## Create or Attach an AmlCompute cluster
We will need to create a compute target for our AutoML run. We will use `vm_size = Standard_D2_V2` in our provisioning configuration and select `max_nodes` to be no greater than 4.

In [4]:
aml_compute_cluster_name = "cpu-cluster"

# Verify that cluster doesn't exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=aml_compute_cluster_name)
    print("Found existing cluster, use it.")

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(workspace=ws,
                                       name=aml_compute_cluster_name,
                                       provisioning_configuration=compute_config)
    
aml_compute.wait_for_completion(show_output=True)

InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
compute_targets = ws.compute_targets

for i, key in enumerate(compute_targets):
    print(f"{i+1}. Compute target\n\tname: {compute_targets[key].name}\n\tType: {compute_targets[key].type}\n")
    

1. Compute target
	name: compute-instance-rvl
	Type: ComputeInstance

2. Compute target
	name: cpu-cluster
	Type: AmlCompute



## Dataset

### Overview

This dataset contains the medical records of 299 patients who had heart failure, collected during their follow-up period, where each patient profile has 13 clinical features.

The 12 clinical input features and the target feature are:

- age: age of the patient (years)
- anaemia: decrease of red blood cells or hemoglobin (boolean)
- high blood pressure: if the patient has hypertension (boolean)
- creatinine phosphokinase (CPK): level of the CPK enzyme in the blood (mcg/L)
- diabetes: if the patient has diabetes (boolean)
- ejection fraction: percentage of blood leaving the heart at each contraction (percentage)
- platelets: platelets in the blood (kiloplatelets/mL)
- sex: woman or man (binary)
- serum creatinine: level of serum creatinine in the blood (mg/dL)
- serum sodium: level of serum sodium in the blood (mEq/L)
- smoking: if the patient smokes or not (boolean)
- time: follow-up period (days)
- [target] death event: if the patient deceased during the follow-up period (boolean)

The task we are concerned with is to predict if the patient deceased during the follow-up period. We will be using `DEATH_EVENT` column as the target and since this is a boolean variable, the task at hand is Binary Classification. 

In the cell below, we write code to access the data that we will be using in this project.

We will try to load the dataset from the Workspace. If it isn't found because it was deleted, it can be recreated with the link that has the CSV

Make sure the `key` is the same name as the dataset that is uploaded. Also we provide a matching description. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them. 


In [6]:
ws.datasets.keys()

KeysView({})

In [7]:
found = False
key = "Health-Failure"
description_text = "Health Failure dataset for mortality prediction"

dataset_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv"

if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("The Dataset was found!")
else:
    # Create AML Dataset and register it into Workspace
    dataset = Dataset.Tabular.from_delimited_files(dataset_url)
    #Register Dataset in Workspace
    dataset = dataset.register(workspace=ws,
                               name=key,
                               description=description_text)

df = dataset.to_pandas_dataframe()

In [8]:
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [10]:
# Split the dataset into training and testing datasets
train_df, test_df = train_test_split(df, test_size=0.2)

# Save training data to csv file
train_df.to_csv("./data/train_data.csv", index=False)

# Read saved training data and create a dataset in Azure ML
data_store = ws.get_default_datastore()
data_store.upload(src_dir="./data", target_path="training_data")
train_ds = TabularDatasetFactory.from_delimited_files(path=[(data_store, 'training_data/train_data.csv')])


Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the `TabularDataset`, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [11]:
train_ds.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,70.0,0,232,0,30,0,173000.00,1.20,132,1,0,210,0
1,57.0,1,129,0,30,0,395000.00,1.00,140,0,0,42,1
2,86.0,0,582,0,38,0,263358.03,1.83,134,0,0,95,1
3,80.0,0,148,1,38,0,149000.00,1.90,144,1,1,23,1
4,45.0,0,582,0,20,1,126000.00,1.60,135,1,0,180,1


## AutoML Configuration

As mentioned above in the dataset section we are dealing with a binary classification. Therefore the argument `task` is set to `classification` and the since we are predicting `DEATH_EVENT` we need to set `label_column_name="DEATH_EVENT"`

To help manage child runs and when they can be performed, we recommend you create a dedicated cluster per experiment, and match the number of `max_concurrent_iterations` of your experiment to the number of nodes in the cluster. This way, you use all the nodes of the cluster at the same time with the number of concurrent child runs/iterations you want.

Configure `max_concurrent_iterations` in your `AutoMLConfig` object. If it is not configured, then by default only one concurrent child run/iteration is allowed per experiment.

Besides other arguments that are self-explanatory, to automate Feature engineering AzureML enables this through `featurization` that needs to be set to `True`. This way features that best characterize the patterns in the data are selected to create predictive models.

In [12]:
# AutoMl settings
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "primary_metric": "AUC_weighted",
    "enable_early_stopping": True,
    "verbosity": logging.INFO
}

# AutoMl config
automl_config = AutoMLConfig(compute_target=aml_compute,
                             task="classification",
                             training_data=train_ds,
                             label_column_name="DEATH_EVENT",
                             n_cross_validations=5,
                             featurization="auto",
                             path=project_folder,
                             debug_log = "automl_errors.log",
                             **automl_settings                             
                            )

In [13]:
# Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment-heart-failure-classification,AutoML_d4513627-f751-4298-b3a2-00020204a9a9,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS

## Run Details

In the cell below, use the `RunDetails` widget to show the different experiments.

In [14]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment-heart-failure-classification,AutoML_d4513627-f751-4298-b3a2-00020204a9a9,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_d4513627-f751-4298-b3a2-00020204a9a9',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-08-24T11:18:21.198965Z',
 'endTimeUtc': '2021-08-24T11:47:48.482985Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-experiment-heart-failure-classification","subscription_id":"b17f1c19-34a2-47b8-a207-40ea477828fc","resource_group":"resource-group-rvl","workspace_name":"workspace-rvl","region":"westeurope","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_m

## Best Model

In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [54]:
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

In [17]:
# Retrieve your best automl model.
best_run, best_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()

In [18]:
best_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=0, reg_alpha=0, reg_lambda=0.10416666666666667, subsample=0.7, tree_method='auto'))], verbose=False))], flatten_transform=None, weights=[0.2, 0.06666666666666667, 0.06666666666666667, 0.13333333333333333, 0.06666666666666667, 0.06666666666666667, 0.13333333333333333, 0.06666666666666667, 0.13333333333333333, 0.06666666666666667]))],
         verbose=False)

In [55]:
print_model(best_model)

datatransformer
{'enable_dnn': False,
 'enable_feature_sweeping': True,
 'feature_sweeping_config': {},
 'feature_sweeping_timeout': 86400,
 'featurization_config': None,
 'force_text_dnn': False,
 'is_cross_validation': True,
 'is_onnx_compatible': False,
 'observer': None,
 'task': 'classification',
 'working_dir': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-instance-rvl/code/Users/vlas'}

prefittedsoftvotingclassifier
{'estimators': ['20', '26', '33', '4', '31', '22', '32', '24', '17', '12'],
 'weights': [0.2,
             0.06666666666666667,
             0.06666666666666667,
             0.13333333333333333,
             0.06666666666666667,
             0.06666666666666667,
             0.13333333333333333,
             0.06666666666666667,
             0.13333333333333333,
             0.06666666666666667]}

20 - standardscalerwrapper
{'class_name': 'StandardScaler',
 'copy': True,
 'module_name': 'sklearn.preprocessing._data',
 'with_mean': False,
 'with_std': Fals

In [19]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment-heart-failure-classification,AutoML_d4513627-f751-4298-b3a2-00020204a9a9_35,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [20]:
best_run_metrics

{'f1_score_weighted': 0.8697670189176063,
 'precision_score_micro': 0.8703014184397162,
 'matthews_correlation': 0.6988607456385812,
 'average_precision_score_weighted': 0.9441935985675087,
 'recall_score_micro': 0.8703014184397162,
 'precision_score_macro': 0.8511901170724702,
 'average_precision_score_micro': 0.9435499333957725,
 'recall_score_macro': 0.8478104482611146,
 'log_loss': 0.36451115239208015,
 'balanced_accuracy': 0.8478104482611146,
 'precision_score_weighted': 0.871563327276719,
 'weighted_accuracy': 0.8854696900597359,
 'f1_score_macro': 0.8481956199258593,
 'recall_score_weighted': 0.8703014184397162,
 'average_precision_score_macro': 0.9270424147844227,
 'f1_score_micro': 0.8703014184397162,
 'accuracy': 0.8703014184397162,
 'norm_macro_recall': 0.6956208965222291,
 'AUC_micro': 0.9401798120693124,
 'AUC_weighted': 0.9351966284181925,
 'AUC_macro': 0.9351966284181925,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_d4513627-f751-4298-b3a2-00020204a9a

In [21]:
best_run.get_details()

{'runId': 'AutoML_d4513627-f751-4298-b3a2-00020204a9a9_35',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-08-24T11:46:04.660356Z',
 'endTimeUtc': '2021-08-24T11:47:45.517912Z',
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'AUC_weighted\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'automl-experiment-heart-failure-classification\',\'compute_target\':\'cpu-cluster\',\'subscription_id\':\'b17f1c19-34a2-47b8-a207-40ea477828fc\',\'region\':\'westeurope\',\'spark_service\':None}","ensemble_run_id":"AutoML_d4513627-f751-4298-b3a2-00020204a9a9_35","experiment_name":"automl-experiment-heart-failure-classification","workspace_name":"work

In [22]:
best_run.get_properties()

{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'AUC_weighted\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'automl-experiment-heart-failure-classification\',\'compute_target\':\'cpu-cluster\',\'subscription_id\':\'b17f1c19-34a2-47b8-a207-40ea477828fc\',\'region\':\'westeurope\',\'spark_service\':None}","ensemble_run_id":"AutoML_d4513627-f751-4298-b3a2-00020204a9a9_35","experiment_name":"automl-experiment-heart-failure-classification","workspace_name":"workspace-rvl","subscription_id":"b17f1c19-34a2-47b8-a207-40ea477828fc","resource_group_name":"resource-group-rvl"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '35',
 '_aml_system_scenario_identifi

In [23]:
# Save the best model
joblib.dump(best_model, filename="./best_automl_model.joblib")

['./best_automl_model.joblib']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In the cell below, register the model, create an inference config and deploy the model as a web service.

In [24]:
model = Model.register(workspace=ws,
                       model_name="heart_failure_pred_model", 
                       model_path="./best_automl_model.joblib",
                       description="Best AutoML model"
                      )

Registering model heart_failure_pred_model


In [25]:
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
env = Environment.from_conda_specification(name="myenv", file_path="./myenv.yml")

#env = best_run.get_environment()

inf_config = InferenceConfig(environment=env,
                             entry_script='./score.py')

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                                       memory_gb=1,
                                                       auth_enabled=False,
                                                       enable_app_insights=True)

service = Model.deploy(workspace=ws, 
                       name="automl-service",
                       models=[model],
                       inference_config=inf_config,
                       deployment_config=deployment_config)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.................
2021-08-24 12:42:01+00:00 Creating Container Registry if not exists..
2021-08-24 12:52:01+00:00 Registering the environment..
2021-08-24 12:52:03+00:00 Building image..
2021-08-24 13:10:08+00:00 Generating deployment configuration.
2021-08-24 13:10:10+00:00 Submitting deployment to compute..
2021-08-24 13:10:24+00:00 Checking the status of deployment automl-service..
2021-08-24 13:13:20+00:00 Checking the status of inference endpoint automl-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


Check the state of the deployed service and get its URIs 

In [26]:
print(f"Service State: {service.state}\n")
print(f"Scoring URI: {service.scoring_uri}\n")
print(f"Swagger URI: {service.swagger_uri}\n")

Service State: Healthy

Scoring URI: http://bb92453d-4ad8-4df2-8c30-c7736042c4f7.westeurope.azurecontainer.io/score

Swagger URI: http://bb92453d-4ad8-4df2-8c30-c7736042c4f7.westeurope.azurecontainer.io/swagger.json



In the cell below, send a request to the web service you deployed to test it.

In [48]:
# URL for the web service
scoring_uri = service.scoring_uri

# If the service is authenticated, set the key or token
#key = ""

# 3 sets of data to score, so we get two results back
data_df = test_df.sample(n=3)
labels = data_df.pop('DEATH_EVENT')


# Convert to JSON string
input_data = json.dumps({"data": data_df.to_dict(orient='records')})
with open("input_data.json", 'w') as _f:
    _f.write(input_data)

print(input_data)

{"data": [{"age": 42.0, "anaemia": 1, "creatinine_phosphokinase": 86, "diabetes": 0, "ejection_fraction": 35, "high_blood_pressure": 0, "platelets": 365000.0, "serum_creatinine": 1.1, "serum_sodium": 139, "sex": 1, "smoking": 1, "time": 201}, {"age": 49.0, "anaemia": 0, "creatinine_phosphokinase": 972, "diabetes": 1, "ejection_fraction": 35, "high_blood_pressure": 1, "platelets": 268000.0, "serum_creatinine": 0.8, "serum_sodium": 130, "sex": 0, "smoking": 0, "time": 187}, {"age": 65.0, "anaemia": 0, "creatinine_phosphokinase": 56, "diabetes": 0, "ejection_fraction": 25, "high_blood_pressure": 0, "platelets": 237000.0, "serum_creatinine": 5.0, "serum_sodium": 130, "sex": 0, "smoking": 0, "time": 207}]}


In [49]:
# Set the content type
headers = {"Content-Type": "application/json"}

# If authentication is enabled, set the authorization header
#headers["Authorization"] = f"Bearer {key}"

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

[0, 0, 0]


In [50]:
print(f"Predictions from Service: {resp.json()}\n")
print(f"Data Labels: {labels.tolist()}")

Predictions from Service: [0, 0, 0]

Data Labels: [0, 0, 0]


In the cell below, print the logs of the web service and delete the service

In [56]:
print(service.get_logs())

2021-08-24T13:13:08,829749570+00:00 - gunicorn/run 
2021-08-24T13:13:08,829591565+00:00 - iot-server/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-08-24T13:13:08,829357557+00:00 - rsyslog/run 
2021-08-24T13:13:08,864606206+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_59fba923920e15fa8fbe872c700ad33f/lib/libuuid.so.1: no version information available (required by rsyslogd)
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (13)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 34
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-08-24T13:13:11,727801275+00:00 - iot-server/finish 1 0
2021-08-24T13:13:11,728874210+00:00 - Exit code 1 is normal. Not restarting iot-server.
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-08-24 13:13:16,323 | root | INFO | Starting up app insights client
logging socket was 

In [57]:
service.delete()